In [3]:
!pip install promptlayer


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [7]:
# Instead of `import openai` we will use
openai = promptlayer.openai
openai.api_key = os.environ.get("OPENAI_API_KEY")

# Make a completion to OpenAI
response = openai.Completion.create(
  engine="text-davinci-003", 
  prompt="My name is",
)
print(response.choices[0].text)


 Maria.
I am 22 years old and from the United Kingdom.




In [8]:
response = openai.Completion.create(
  engine="text-davinci-003", 
  prompt="My name is",
  pl_tags=["getting_started_example"] # 🍰 PromptLayer tags
)

print(response.choices[0].text)


 Ashley and I have 3 years of experience as a back-end developer. I


In [5]:
import promptlayer
import os
promptlayer.api_key = os.environ.get("PROMPTLAYER_API_KEY")

# Swap out your 'import openai'
openai = promptlayer.openai
openai.api_key = os.environ.get("OPENAI_API_KEY")

# Do something fun 🚀
openai.Completion.create(
  engine="text-ada-001", 
  prompt="My name is", 
  pl_tags=["name-guessing", "pipeline-2"]
)

<OpenAIObject text_completion id=cmpl-8IJeXo0GJxpaglGmVXegnCNciM1oZ at 0x11c374f40> JSON: {
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-8IJeXo0GJxpaglGmVXegnCNciM1oZ",
  "object": "text_completion",
  "created": 1699376141,
  "model": "text-ada-001",
  "choices": [
    {
      "text": " capitalized\n\nHaley\n\nHello Haley!",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 3,
    "completion_tokens": 11,
    "total_tokens": 14
  }
}

In [9]:
response, pl_request_id = openai.Completion.create(
  engine="text-davinci-003", 
  prompt="What is the capital of New York? \\n\\nThe capital of New York is",
  pl_tags=["getting_started_example"],
  return_pl_id=True # Make sure to set this to True
)

answer = response.choices[0].text
print(answer)
correct_answer = "albany" in answer.lower()

# Log score to 🍰 PromptLayer
promptlayer.track.score(
    request_id=pl_request_id,
    score=100 if correct_answer else 0,
)


 Albany.


True

In [10]:
prompt_template = """You are an AI assistant that helps travelers pick a city to travel to. 
You do this by rating how much a person would enjoy a city based on their interests.
Given a city and interests, you respond with an integer 1-10 where 10 is the most enjoyment and 0 is the least.

Sample city: New York City
Sample interests: food, museums, hiking
Sample answer: 8

City: {city}
Interests: {interests}
Answer: """

response, pl_request_id = openai.Completion.create(
  engine="text-davinci-003", 
  prompt=prompt_template.format(city="Washington, D.C.", interests="resorts, museums, beaches"),
  pl_tags=["getting_started_example"],
  return_pl_id=True
)

answer = response.choices[0].text
print(answer)

# Let's convert the answer to an int
numeric_answer = None
error_message = None
try:
    numeric_answer = int(answer.strip())
except ValueError as e:
    error_message = str(e)
    pass

# Use score in 🍰 PromptLayer to track if answer was an int
promptlayer.track.score(
    request_id=pl_request_id,
    score=100 if numeric_answer else 0,
)

print("Numeric answer:", numeric_answer)

# Log metadata for request in 🍰 PromptLayer
promptlayer.track.metadata(
    request_id=pl_request_id,
    metadata={
        "referrer": "getting_started.ipynb",
        "origin": "NYC, USA",
        "user_id": "sdf328",
        "error_message": "No error" if numeric_answer else error_message,
    }
)


 7
Numeric answer: 7


True

In [11]:
city_choice_prompt = promptlayer.prompts.get("city_choice")
city_choice_prompt_v1 = promptlayer.prompts.get("city_choice", version=1)
city_choice_prompt_prod = promptlayer.prompts.get("city_choice", label="prod")
print(city_choice_prompt_v1['template'])


Exception: PromptLayer had the following error while getting your prompt: Prompt not found

In [12]:
import promptlayer # Don't forget this 🍰
from langchain.callbacks import PromptLayerCallbackHandler

# OpenAI Completion Model
from langchain.llms import OpenAI

llm = OpenAI(
    model_name="text-davinci-003",
    callbacks=[
        PromptLayerCallbackHandler(pl_tags=["langchain"])
    ],
)
llm("How tall are you?")


'\n\nI am 5\'7" tall.'